In [ ]:
import json

with open("codigo_civil.json", "r") as f:
    cod_civil = json.load(f)

In [ ]:
def collect_artigo_and_header(data):
    result = {'articles': [], 'contexts': []}

    def traverse(data):
        if isinstance(data, dict):
            for key, value in data.items():
                if key == 'artigos' and value:
                    result['articles'].append(value)
                if key == 'header' and value:
                    result['contexts'].append(value)
                # Recursive call to handle nested dictionaries
                if isinstance(value, dict):
                    traverse(value)
                elif isinstance(value, list):
                    traverse(value)
        elif isinstance(data, list):
            for item in data:
                # Recursive call to handle lists of dictionaries
                traverse(item)

    traverse(data)
    return result

result = collect_artigo_and_header(cod_civil['parte especial']['LIVRO IV'])
iv_context = ''.join(result['contexts'])
iv_articles = result['articles']

result = collect_artigo_and_header(cod_civil['parte especial']['LIVRO V'])
v_context = ''.join(result['contexts'])
v_articles = result['articles']

In [ ]:
def get_article_group(article_text, articles_library=iv_articles+v_articles):
    groups = []
    for group in articles_library:
        for article in group:
            if article_text in article:
                groups.append(group)
                break
    return groups

def get_articles(article_text, articles_library=iv_articles+v_articles):
    articles = []
    for group in articles_library:
        for article in group:
            if article_text in article:
                articles.append(article)
    return articles

get_articles("1.523")

['Art. 1.523. Não devem casar:\n\nI - o viúvo ou a viúva que tiver filho do cônjuge falecido, enquanto não fizer inventário dos bens do casal e der partilha aos herdeiros;\n\nII - a viúva, ou a mulher cujo casamento se desfez por ser nulo ou ter sido anulado, até dez meses depois do começo da viuvez, ou da dissolução da sociedade conjugal;\n\nIII - o divorciado, enquanto não houver sido homologada ou decidida a partilha dos bens do casal;\n\nIV - o tutor ou o curador e os seus descendentes, ascendentes, irmãos, cunhados ou sobrinhos, com a pessoa tutelada ou curatelada, enquanto não cessar a tutela ou curatela, e não estiverem saldadas as respectivas contas.\n\nParágrafo único. É permitido aos nubentes solicitar ao juiz que não lhes sejam aplicadas as causas suspensivas previstas nos incisos I, III e IV deste artigo, provando-se a inexistência de prejuízo, respectivamente, para o herdeiro, para o ex-cônjuge e para a pessoa tutelada ou curatelada; no caso do inciso II, a nubente deverá 

In [29]:
def get_article_id(sample):
    arts = sample['artigos']
    arts = arts.split(',')
    for idx, art in enumerate(arts):
        if ('inciso' in art.lower()) or ('parágrafo' in art.lower()) or ('§' in art):
            arts.pop(idx)
            continue
        if ' e ' in art.lower():
            arts[idx] = art.split(' e ')[0].strip()
            arts.insert(idx+1, art.split(' e ')[1].strip())
        else:
            arts[idx] = art.strip()
    return arts

In [30]:
import pandas as pd
df = pd.read_csv('/content/LLM1_QA_generation_clean2.csv')

In [39]:
get_article_id(df.iloc[224])

['Art. 1.783-A']

In [43]:
x = get_article_id(df.iloc[68])
art_text = '\n'.join(['\n'.join(get_articles(art)) for art in x])
art_text

'Art. 1.969. O testamento pode ser revogado pelo mesmo modo e forma como pode ser feito.\n\n  \nArt. 1.970. A revogação do testamento pode ser total ou parcial.\n\nParágrafo único. Se parcial, ou se o testamento posterior não contiver cláusula revogatória expressa, o anterior subsiste em tudo que não for contrário ao posterior.\n\n  \nArt. 1.971. A revogação produzirá seus efeitos, ainda quando o testamento, que a encerra, vier a caducar por exclusão, incapacidade ou renúncia do herdeiro nele nomeado; não valerá, se o testamento revogatório for anulado por omissão ou infração de solenidades essenciais ou por vícios intrínsecos.\n\n  '